In [1]:
import pysam
import os
import pandas as pd
import numpy as np

import pybedtools as pb

bam_file = "/paedyl01/disk1/yangyxt/wgs/GIAB_samples/aligned_results/HG007.SD.deduped.bam"

os.chdir("/home/louisshe/work/SDrecall/")

pb.set_bedtools_path("/software/bedtools/2.30.0/bin")

## Actual use case

In [2]:
import os
import logging

from multialign import get_multialign_regions, bam_subset
from build_sd import build_sd_pairs

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger("root")

logger.info(f"Working dir: {os.getcwd()}")

# merged_depth = get_multialign_regions(bam_file, target_region="smn_hg19.bed")

# # # This step creates selected_regions.json
# bam_subset(bam_file, mq_cutoff = 20)

2024-01-16 19:36:36,584 - INFO - Working dir: /home/louisshe/work/SDrecall


## Test run

In [37]:
df = pd.read_json("selected_regions.json", lines=True)
sd_map = build_sd_pairs(bam_file, confidence=0.95, threads=8, full=True)


/home/louisshe/work/SDrecall/build_sd.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pnext_upstream["end_pos"] = pnext_upstream["pnext"] + pnext_upstream["tlen"].abs()
/home/louisshe/work/SDrecall/build_sd.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pnext_downstream["end_pos"] = pnext_downstream["pos"] + pnext_downstream["tlen"].abs()


In [65]:
reference_map = "WGAC.hg19.cigar.trimmed.homo.expanded.bed"

reference_sd, sd_map = pb.BedTool(reference_map).sort(), pb.BedTool.from_dataframe(sd_map).sort()
logger.info(f"Total coverage of reference SD map = {reference_sd.total_coverage()}bp")


2024-01-17 11:53:52,078 - INFO - Total coverage of reference SD map = 132239799bp


<BedTool(/tmp/pbs.1411867.omics/pybedtools.2rmn8gxm.tmp)>

In [82]:
overlapped_sd = reference_sd.intersect(sd_map, wo=True).to_dataframe(disable_auto_names=True,
                                                    names = ["chr_1", "start_1", "end_1",
                                                             "chr_2", "start_2", "end_2",
                                                             "strand1", "strand2",
                                                             "cigar", "mismatch_rate",
                                                             "chr_bam1", "start_bam1", "end_bam1",
                                                             "chr_bam2", "start_bam2", "end_bam2",
                                                             "overlap_len"]) \
                                           .loc[:,["chr_1", "start_1", "end_1", "strand1",
                                                   "chr_2", "start_2", "end_2", "strand2",
                                                   "chr_bam1", "start_bam1", "end_bam1"]] \
                                           .drop_duplicates().dropna()

In [83]:
overlapped_sd.loc[(overlapped_sd["strand1"] == "-") & (overlapped_sd["strand2"] == "-"), ["strand1", "strand2"]] = "+"

In [88]:
for _, grp_df in overlapped_sd.groupby(["chr_bam1", "start_bam1", "end_bam1"], as_index=False):
    print(grp_df.shape)
    # sys.exit()

(4, 11)
(6, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(6, 11)
(5, 11)
(6, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(4, 11)
(1, 11)
(1, 11)
(1, 11)
(1, 11)
(1, 11)
(1, 11)
(1, 11)
(1, 11)
(3, 11)
(3, 11)
(3, 11)
(4, 11)
(4, 11)
(4, 11)
(3, 11)
(3, 11)
(3, 11)
(4, 11)
(5, 11)
(3, 11)
(2, 11)
(2, 11)
(2, 11)
(2, 11)
(2, 11)
(2, 11)
(2, 11)
(2, 11)
(2, 11)
(2, 11)
(1, 11)
(1, 11)
(1, 11)
(1, 11)
(1, 11)
(2, 11)
(2, 11)
(3, 11)
(3, 11)
(3, 11)
(3, 11)
(3, 11)
(3, 11)
(3, 11)
(3, 11)
(3, 11)
(3, 11)
(3, 11)
(4, 11)
